In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin




In [2]:

# creating the class
class TitanicCleaning(pd.DataFrame):
    def normalize_cols(self):
        '''
        cleaning columns
        '''
        self.columns = self.columns.str.replace(" ", "_")
        self.columns = self.columns.str.lower()
        return(self)

    def drop_irrel_cols(self):
        '''for this kaggle titanic_cleaning dataset, we determined that
        we can fill nulls with zeros.'''
        self =  self.drop(["cabin","ticket","age"], axis = 1)
        return self


class TitanicMl(TitanicCleaning, BaseEstimator, TransformerMixin):

    def brfc_shortcut(self, y):
        '''Class to simplfy Balanced
        Random Forrest Classifier. Enter prediction column, 
        and then the features for training the model.
        
        Returns balanced_accuracy_score, confusion matrix,
        classification_report_imbalanced'''
        # features = str[args]
        y = self[y]

        features = self[["pclass", "sex", "sibsp", "parch","fare"]]

        # X is our data for the ml model. 
        X = pd.get_dummies(self[features])


        # X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


        # random_forest = BalancedRandomForestClassifier(n_estimators = 100)
        # random_forest = random_forest.fit(X_train, y_train)

        # # prediction
        # y_pred = random_forest.predict(X_test)

        # #reporting 
        # bas = balanced_accuracy_score(y_test, y_pred)
        # cm = confusion_matrix(y_test, y_pred)
        # cri = classification_report_imbalanced(y_test, y_pred)

        # return(bas,cm,cri)
        return(X)

In [3]:
# Reading in Data
test_df = pd.read_csv(os.path.join("resources","test.csv"))
train_df = pd.read_csv(os.path.join("resources","train.csv"))

parch = # of parents / children aboard the Titanic
<br>
sibsp = # of siblings / spouses aboard the Titanic
<br>
survival = {0 = No, 1 = Yes}

In [4]:
# Normalizing Columns
train_df = TitanicCleaning(train_df)
train_df = train_df.normalize_cols()


test_df = TitanicCleaning(test_df)
test_df = test_df.normalize_cols()


#dropping cabin and ticket column, as they may confuse the model. 
train_df = train_df.drop_irrel_cols()
train_df.head()




,passengerid,survived,pclass,name,sex,sibsp,parch,fare,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,0,0,8.0500,S


In [5]:
# Getting our ml specific data. 
# This will be what we are using our ml model to train with. 
y = train_df.survived

features = ["pclass", "sex", "sibsp", "parch","fare"]

# X is our data for the ml model. 
X = pd.get_dummies(train_df[features])


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

y_train.value_counts()

0    421
1    247
Name: survived, dtype: int64

# Balanced Random Forest Classifier

In [6]:
# creating the model. 
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

# prediction
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8166118421052632

In [7]:
# printing confusion matrix
confusion_matrix(y_test, y_pred)




array([[108,  20],
       [ 20,  75]])

In [10]:
# export submission form
output = pd.DataFrame({'PassengerId': test_df.passengerid, 'Survived': y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
output.shape

ValueError: array length 223 does not match index length 418

Without "age" acc = 0.7969161184210527

with dropped null "age" = 0.8092350746268657

In [8]:
ml_df = TitanicMl(train_df)
ml_df = ml_df.normalize_cols()

features = ["pclass", "sex", "sibsp", "parch","fare"]
ml_df.brfc_shortcut("survived")


ValueError: Boolean array expected for the condition, not int64